# Preparation

In [2]:
# Input Here

import pandas as pd

QUIZ_DF = pd.read_csv("quiz\InteractivePlottingQuiz-all_0.csv") # Quiz CSV Path

SCORE_ACADEMY_LINK = 'https://docs.google.com/spreadsheets/d/1cGJ0pn9k9gKCBnceWVwaL9D7BBDMNjLh8uPYlaBlJi8/edit?usp=sharing' # Score Academy Link
NAMA_SHEET = "Academy: Batch 23" # Sheet Name (Wizard) 

Import required dependencies to use Google's API

In [3]:
import os.path

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

Define the accesses to be granted

In [4]:
SCOPES = ['https://www.googleapis.com/auth/classroom.courses.readonly', # View your Google Classroom classes.
          'https://www.googleapis.com/auth/classroom.rosters', # Manage your Google Classroom class rosters.
          'https://www.googleapis.com/auth/classroom.profile.emails', # View the email addresses of people in your classes.
          'https://www.googleapis.com/auth/classroom.topics', #See, create, and edit topics in Google Classroom.
          'https://www.googleapis.com/auth/classroom.coursework.students', # Manage coursework and grades for students
          'https://www.googleapis.com/auth/classroom.courseworkmaterials', # See, edit, and create classwork materials in Google Classroom.
          'https://www.googleapis.com/auth/spreadsheets'] # See all your Google Sheets spreadsheets.

Use the provided `token.json` file to authenticate and access user data, if `token.json` is not available, it would be created from the provided `credentials.json` by logging into user's Google Account

In [5]:
creds = None

if os.path.exists('token.json'):
    creds = Credentials.from_authorized_user_file('token.json', SCOPES)
# If there are no (valid) credentials available, let the user log in.
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file(
            'credentials.json', SCOPES)
        creds = flow.run_local_server(port=0)
    # Save the credentials for the next run
    with open('token.json', 'w') as token:
        token.write(creds.to_json())

# Score Academy

In the section, the user is going to access the scores that have been entered on the spreadsheet `Score Academy`

Input the link to the Score Academy spreadsheet and retrieve the Spreadsheet ID

In [6]:
SCORE_ACADEMY_ID = SCORE_ACADEMY_LINK.split(sep='/')[-2]

Specify the sheet and the cell ranges that is going to be accessed

In [7]:
GRADE_RANGE = [f'{NAMA_SHEET}!D:E', f'{NAMA_SHEET}!F:H', f'{NAMA_SHEET}!M:R']

Call the Google Spreadsheet API and retrieve the values of the ranges that have been specified

In [8]:
try:
    service = build('sheets', 'v4', credentials=creds)

    sheet = service.spreadsheets().values().batchGet(spreadsheetId=SCORE_ACADEMY_ID,
                                                     ranges=GRADE_RANGE).execute()
    values = sheet.get('valueRanges', [])
        
except HttpError as error:
    print(error)

Concat the retrieved values as a dataframe

In [10]:
import pandas as pd

email = pd.DataFrame(values[0].get('values'))
grade_dv = pd.DataFrame(values[1].get('values'))
grade_ml = pd.DataFrame(values[2].get('values'))

df = pd.concat([email, grade_dv, grade_ml], axis=1)
df.columns = df.iloc[0]
df.drop(index=0, inplace=True)
df['Email'] = df['Email'].str.lower() 

df

,Email,Email Classroom,P4DS-PS Quiz,DV Quiz,IP Quiz,RM Quiz,C1 Quiz,C2 Quiz,UL Quiz,TS Quiz,NN Quiz
1,danadyaksawijaya@gmail.com,danadyaksawijaya@gmail.com,4,2,None,NaN,NaN,NaN,NaN,NaN,NaN
2,berlianahar@gmail.com,berlianahar@gmail.com,4,2,None,NaN,NaN,NaN,NaN,NaN,NaN
3,angiegrace35@gmail.com,angiegrace35@gmail.com,4,2,None,NaN,NaN,NaN,NaN,NaN,NaN
4,ihsan.azmi.rabbani@gmail.com,ihsan.azmi.rabbani@gmail.com,4,2,None,NaN,NaN,NaN,NaN,NaN,NaN
5,lhpratama2@gmail.com,lhpratama2@gmail.com,4,2,None,NaN,NaN,NaN,NaN,NaN,NaN
6,melissa.offc@gmail.com,melissa.offc@gmail.com,4,2,None,NaN,NaN,NaN,NaN,NaN,NaN
7,nadyanaftalia@gmail.com,nadyanaftalia@gmail.com,4,2,None,NaN,NaN,NaN,NaN,NaN,NaN
8,brenda.thadete@live.com,brenda.thadete@gmail.com,4,2,None,NaN,NaN,NaN,NaN,NaN,NaN
9,ghozyaufan@gmail.com,ghozyaufan@gmail.com,4,2,None,NaN,NaN,NaN,NaN,NaN,NaN
10,kambu.charles@aol.com,kambu.charles@gmail.com,4,0,None,NaN,NaN,NaN,NaN,NaN,NaN


# Course
In this section, the user is going to choose which Google Classroom Course that is going to be accessed

Call the Google Classroom API to access various methods with user's access from the credential that has been authenticated

In [11]:
service = build('classroom', 'v1', credentials=creds)

Use the `courses().list()` method to show a list of the user's courses

In [12]:
results = service.courses().list(pageSize=5).execute()
courses = results.get('courses', [])

if not courses:
    print('No courses found.')

# Prints the names of the first 5 courses.
print('Courses:')
for course in courses:
    print(f"- {course['name']}")

Courses:
- DSS: Data Science Fundamentals
- DSS: Geospatial Data Visualization with Rayshader In R
- Wizard Machine Learning
- Wizard Data Visualization
- Kementerian BUMN


Input the course name (**case-insensitive**) that is going to be accessed (Example: `wizard data visualization`). If the course was found, the course's `id` will be retrieved.

In [13]:
course_input = input("Enter Google Classroom Class Name : ")
course_lowercase = course_input.lower()
course_id = None

for course in courses:
    if course_lowercase == course['name'].lower():
        course_id = course['id']
        break

if course_id == None:
    raise Exception(f"{course_input} course not found")

else:
    print(f'{course_input} found with ID {course_id}')

Wizard Data Visualization found with ID 621398536690


# Quiz
In this section, the grading process is going to be done using the score retrieved from the spreadsheet

Input the quiz code (**case-insensitive**) that is going to be accessed. If the quiz was found, the quiz's `id` will be retrieved. **Quiz Input :**

 - `P4DS` : 1. Q: Programming for Data Science (P4DS) & Practical Statistic (PS)
 - `DV` : 2. Q: Data Visualization (DV)
 - `IP` : 3. Q: Interactive Plotting (IP)
 - `RM`
 - `C1`
 - `C2`
 - `UL`
 - `TS`
 - `NN`

In [14]:
from assets import classcode, quizcode, max_score, quiz_range

service = build('classroom', 'v1', credentials=creds)
response = service.courses().courseWork().list(courseId=course_id).execute()
classworks = response.get('courseWork')

while response.get('nextPageToken'):
    response = service.courses().students().list(courseId=course_id, pageToken = response['nextPageToken']).execute()
    classworks.extend(response.get('courseWork'))

quiz_input = input("Enter Quiz Code: ")
quiz_id = None

for classwork in classworks:
    if classwork['title'] == classcode(quiz_input):
        quiz_id = classwork['id']
        break

if quiz_id == None:
    raise Exception(f"Quiz not found")
else:
    print(f"{classwork['title']} Quiz was found")


3. Q: Interactive Plotting (IP) Quiz was found


# Write Score Academy

In this section, the received grade is going to be written in Score Academy

In [15]:
QUIZ_DF['USER EMAIL'] = QUIZ_DF['USER EMAIL'].str.strip().str.lower()
QUIZ_DF['PASSED STATUS'] = QUIZ_DF['PASSED STATUS'].str.strip().str.lower()
QUIZ_DF = QUIZ_DF[QUIZ_DF['PASSED STATUS'] == "yes"]
QUIZ_DF = QUIZ_DF.drop_duplicates("USER EMAIL")
passed_email = QUIZ_DF["USER EMAIL"]

passed_email

0        kunciran_sakti@yahoo.co.uk
1         theophilos.rico@gmail.com
2             kambu.charles@aol.com
3              ghozyaufan@gmail.com
4      ihsan.azmi.rabbani@gmail.com
                   ...             
665          hjantonius21@gmail.com
666      joshuaevan010196@gmail.com
667        doni.el.tobing@gmail.com
668     tommy.william1297@gmail.com
672                david@algorit.ma
Name: USER EMAIL, Length: 579, dtype: object

In [16]:
df.loc[df["Email"].isin(passed_email.values), quizcode(quiz_input)] = max_score(quiz_input)
df.loc[~df["Email"].isin(passed_email.values), quizcode(quiz_input)] = 0

df

,Email,Email Classroom,P4DS-PS Quiz,DV Quiz,IP Quiz,RM Quiz,C1 Quiz,C2 Quiz,UL Quiz,TS Quiz,NN Quiz
1,danadyaksawijaya@gmail.com,danadyaksawijaya@gmail.com,4,2,1,NaN,NaN,NaN,NaN,NaN,NaN
2,berlianahar@gmail.com,berlianahar@gmail.com,4,2,1,NaN,NaN,NaN,NaN,NaN,NaN
3,angiegrace35@gmail.com,angiegrace35@gmail.com,4,2,1,NaN,NaN,NaN,NaN,NaN,NaN
4,ihsan.azmi.rabbani@gmail.com,ihsan.azmi.rabbani@gmail.com,4,2,1,NaN,NaN,NaN,NaN,NaN,NaN
5,lhpratama2@gmail.com,lhpratama2@gmail.com,4,2,1,NaN,NaN,NaN,NaN,NaN,NaN
6,melissa.offc@gmail.com,melissa.offc@gmail.com,4,2,1,NaN,NaN,NaN,NaN,NaN,NaN
7,nadyanaftalia@gmail.com,nadyanaftalia@gmail.com,4,2,1,NaN,NaN,NaN,NaN,NaN,NaN
8,brenda.thadete@live.com,brenda.thadete@gmail.com,4,2,1,NaN,NaN,NaN,NaN,NaN,NaN
9,ghozyaufan@gmail.com,ghozyaufan@gmail.com,4,2,1,NaN,NaN,NaN,NaN,NaN,NaN
10,kambu.charles@aol.com,kambu.charles@gmail.com,4,0,1,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
try:
    service = build('sheets', 'v4', credentials=creds)
    
    values = [[x] for x in df[quizcode(quiz_input)].values.tolist()]

    body = {
        'values': values
    }
    result = service.spreadsheets().values().update(
        spreadsheetId=SCORE_ACADEMY_ID, range=f'{NAMA_SHEET}!{quiz_range(quiz_input)}',
        valueInputOption="USER_ENTERED", body=body).execute()
    print(f"{result.get('updatedCells')} cells updated.")

except HttpError as error:
    print(f"An error occurred: {error}")

47 cells updated.


### Draft Grades

Submissions from the chosen quiz are going to be graded as **Draft Grade** 

Use the `courses().courseWork().studentSubmissions().list()` method to store a list of the quiz's submissions

In [18]:
submissions = []

service = build('classroom', 'v1', credentials=creds)
response = service.courses().courseWork().studentSubmissions().list(
    courseId=course_id,
    courseWorkId=quiz_id).execute()
submissions.extend(response.get('studentSubmissions', []))

while response.get('nextPageToken'):
    response = service.courses().courseWork().studentSubmissions().list(
        courseId=course_id,
        courseWorkId=quiz_id,
        pageToken = response['nextPageToken']).execute()
    submissions.extend(response.get('studentSubmissions'))

All the stored submission are graded as draft in accordance with the student's e-mail in the Google Classroom using `courses().courseWork().studentSubmissions().patch()` method

In [20]:
from assets import quizcode

draft = []
quiz_code = quizcode(quiz_input)

for submission in submissions:
  # Retrieve student's email
  submission_profile = service.courses().students().get(courseId=course_id, userId=submission['userId']).execute()
  student_df = df.loc[df['Email Classroom'] == submission_profile['profile']['emailAddress']]
  
  # Retrieve student's grade
  if not student_df.empty:
    if isinstance(student_df[quiz_code].values[0], int) or student_df[quiz_code].values[0].isnumeric():
      submission_grade = student_df[quiz_code].values[0]
    else:
      print(f"WARNING: {submission_profile['profile']['name']['fullName']} ({submission_profile['profile']['emailAddress']}) has no grade")
      draft.append([submission_profile['profile']['name']['fullName'], submission_profile['profile']['emailAddress'], None, "NO GRADE"])
      continue      
  else:
    print(f"WARNING: {submission_profile['profile']['name']['fullName']} ({submission_profile['profile']['emailAddress']}) was not found")
    draft.append([submission_profile['profile']['name']['fullName'], submission_profile['profile']['emailAddress'], None, "NOT FOUND"])
    continue

  # Grade the submission as draftGrade
  studentSubmission = {
    'draftGrade': str(submission_grade)
  }

  response = service.courses().courseWork().studentSubmissions().patch(
      courseId=course_id,
      courseWorkId=classwork['id'],
      id=submission['id'],
      updateMask='draftGrade',
      body = studentSubmission ).execute()
  
  draft.append([submission_profile['profile']['name']['fullName'], submission_profile['profile']['emailAddress'], submission_grade, "GRADED"])

draft_df = pd.DataFrame(draft, columns=['Name', 'Email Classroom', 'Grade', 'Status'])
draft_df.sort_values('Status')

,Name,Email Classroom,Grade,Status
0,Nissa Corlidea,ncorlidea99@gmail.com,1.0,GRADED
25,Dimas Aditya,senddimasadit@gmail.com,0.0,GRADED
27,Jayadi butar-butar,jayadidetormentor@gmail.com,1.0,GRADED
53,Louis Sirait,louis.sirait@gmail.com,0.0,GRADED
52,satyo pradana,satyopradana1@gmail.com,1.0,GRADED
51,Charles Kambu,kambu.charles@gmail.com,1.0,GRADED
50,Husna Aydadenta,haydnta@gmail.com,0.0,GRADED
33,Nia Dewi Nofianti,niadewinofianti18@gmail.com,1.0,GRADED
34,Audi Chandra,audichandra94@gmail.com,1.0,GRADED
35,Farhan Zhafar Ramadhan,zhafarfarhan.work@gmail.com,1.0,GRADED


Confirm Draft Grade before continuing

In [18]:
confirmation = input('Return grades? [y]/[n] : ')
if confirmation.lower() == 'y':
    print("Returning student's grades")
else :
    raise Exception("Return cancelled")

Returning student's grades


### Return Grades

Submissions from the chosen quiz are going to be graded as **Assigned Grade**

All the stored submission are grades are **returned** in accordance with the student's e-mail in the Google Classroom using `courses().courseWork().studentSubmissions().patch()` method

In [22]:
try:
    for submission in submissions:
        # Retrieve student's email
        submission_profile = service.courses().students().get(courseId=course_id, userId=submission['userId']).execute()
        student_df = df.loc[df['Email Classroom'] == submission_profile['profile']['emailAddress']]
        
        # Retrieve student's grade
        if not student_df.empty:
            if isinstance(student_df[quiz_code].values[0], int) or student_df[quiz_code].values[0].isnumeric():
                submission_grade = student_df[quiz_code].values[0]
            else:
                continue
        else:
            continue
        
        # Grade the submission as assignedGrade
        studentSubmission = {
            'assignedGrade': str(submission_grade)
        }

        response = service.courses().courseWork().studentSubmissions().patch(
            courseId=course_id,
            courseWorkId=classwork['id'],
            id=submission['id'],
            updateMask='assignedGrade',
            body = studentSubmission).execute()
        
    print('Quiz grades was successfully returned')

except HttpError as error:
    print(error)

Quiz grades was successfully returned
